<a href="https://www.kaggle.com/code/mandu5/financial-advisor?scriptVersionId=197124922" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
!pip install -q -U -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
df = pd.read_csv('/kaggle/input/finance-data/Finance_data.csv')

In [ ]:
def create_input(row):
    profile = (
        f"User Profile:\n"
        f"- Gender: {row['gender']}\n"
        f"- Age: {row['age']}\n"
        f"- Investment Avenues Interested: {row['Investment_Avenues']}\n"
        f"- Preferred Investments:\n"
    )
    # List of investments with their preferences
    investments = []
    investment_columns = ['Mutual_Funds', 'Equity_Market', 'Debentures', 'Government_Bonds',
                          'Fixed_Deposits', 'PPF', 'Gold']
    for col in investment_columns:
        preference = row[col]
        investments.append(f"  - {col.replace('_', ' ')} (Preference: {preference})")
    profile += '\n'.join(investments) + '\n'
    profile += (
        f"- Investment Objectives: {row['Objective']}\n"
        f"- Investment Purpose: {row['Purpose']}\n"
        f"- Investment Duration: {row['Duration']}\n"
        f"- Expected Returns: {row['Expect']}\n"
        f"- Savings Objective: {row['What are your savings objectives?']}\n"
        f"- Source of Information: {row['Source']}\n\n"
        f"Question:\n"
        f"What investment strategies should I consider?"
    )
    return profile

def create_output(row):
    # Construct an output string providing financial advice
    output = (
        f"Considering your objectives of {row['Objective']} and {row['Purpose']} over {row['Duration']}, "
        f"you might explore investment avenues like {row['Avenue']}. "
        f"Given your expected returns of {row['Expect']}, these options align with your goals. "
        f"Remember to diversify your portfolio and assess the risks involved. "
        f"Consulting a financial advisor can provide personalized guidance."
    )
    return output

# Create 'input' column
df['input'] = df.apply(create_input, axis=1)

# Create 'output' column
df['output'] = df.apply(create_output, axis=1)

# Combine 'input' and 'output' into 'text' column
df['text'] = df.apply(lambda row: f"input: {row['input']}\noutput: {row['output']}", axis=1)

In [ ]:
# Remove rows with missing values
df.dropna(subset=['input', 'output'], inplace=True)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Prepare the text column for training
df['text'] = df.apply(lambda row: f"input: {row['input']}\noutput: {row['output']}", axis=1)

In [ ]:
# Display an example input-output pair
example_row = df.iloc[0]
print("Input:")
print(example_row['input'])
print("\nOutput:")
print(example_row['output'])

In [ ]:
from datasets import Dataset

# Remove any rows with missing values in 'text'
df.dropna(subset=['text'], inplace=True)

# Create Dataset from pandas DataFrame
train_data = Dataset.from_pandas(df[['text']])

# Verify the Dataset
print(train_data)

In [ ]:
# Initialize Model
model_name = "/kaggle/input/gemma/transformers/2b-it/3"
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    low_cpu_mem_usage=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model.config.hidden_activation = 'gelu_pytorch_tanh'

max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)

In [ ]:
# Fine-Tune the Model
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="financial_advisor_model",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

# Start training
trainer.train()

In [ ]:
# Save and Merge Model
trainer.save_model()
tokenizer.save_pretrained("financial_advisor_model")

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
     "financial_advisor_model",
     torch_dtype=compute_dtype,
     device_map="auto",
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./financial_advisor_pretrained", safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./financial_advisor_pretrained")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Specify the path to your pretrained model directory
model_name = "./financial_advisor_pretrained"

# Load the model without device_map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to generate advice
def get_financial_advice(user_profile, model=model, tokenizer=tokenizer):
    prompt = f"input: {user_profile}\noutput:"
    # Tokenize the input and move tensors to the same device as the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        no_repeat_ngram_size=2,
        early_stopping=True,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split('output:')[-1].strip()

# Example usage
example_input = df.iloc[0]['input']
print("User Profile and Question:")
print(example_input)
print("\nGenerated Advice:")
print(get_financial_advice(example_input))